In [1]:
import numpy as np
np.random.seed(777)
import time
import keras as keras
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import decode_predictions
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten
from keras.layers import merge,Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception
from keras.applications.vgg16 import preprocess_input as pi_vgg16
from keras.applications.inception_v3 import preprocess_input as pi_incep
from keras.applications.resnet50 import preprocess_input as pi_resnet
from keras.applications.vgg19 import preprocess_input as pi_vgg19
from keras.applications.xception import preprocess_input as pi_xcep 
from keras.models import load_model
from numpy import array
from numpy import argmax
from sklearn.metrics import accuracy_score
from  numpy import mean 
from numpy import std
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Concatenate
from keras.layers import BatchNormalization,Dropout
from keras.regularizers import l1,l2,l1_l2
import math
from keras import backend as K
from keras.metrics import categorical_accuracy
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import os
from os.path import basename, join, exists

In [3]:
def run_predictions(image_path):
    loaded_model=keras.models.load_model('xception_upto95frozen.h5',compile=False)
    loaded_model.load_weights('xception_upto95frozen_weights.h5')
    test_dir=image_path
    
    img_height =512
    img_width = 512
    batch_size =32
    input_shape = (img_width, img_height, 3)
    
    random_seed = np.random.seed(1142)
    test_datagen=ImageDataGenerator(rescale=1./255)
    test_generator_xcep=test_datagen.flow_from_directory(test_dir,
                                                      target_size=(img_height, img_width),
                                                          batch_size=batch_size, 
                                                          seed=random_seed,
                                                          shuffle=False,
                                                        class_mode='categorical')
    
    nb_test_samples = len(test_generator_xcep.filenames)
    predict_size_test = int(math.ceil(nb_test_samples / batch_size))
    
    #Xception
    model = Xception(include_top=False, weights="imagenet",pooling='avg',input_tensor=Input(shape=input_shape))
    
    image_input =model.input
    x1 = GlobalAveragePooling2D()(model.get_layer("block4_sepconv1_act").output) #layer_26
    x2 = GlobalAveragePooling2D()(model.get_layer("block5_sepconv1_act").output) #layer_36
    x3 = GlobalAveragePooling2D()(model.get_layer("block14_sepconv1").output)  #layer_126
    out= Concatenate()([x1,x2,x3])
    custom_xcep_model = Model(image_input , out)
    
    for layer in custom_xcep_model.layers[:95]:
        layer.trainable = False
        
        
    #Saving features of the image
    bottleneck_features = custom_xcep_model.predict_generator(test_generator_xcep, predict_size_test)
    np.save('bottleneck_features.npy',bottleneck_features)
    
    data=np.load('bottleneck_features.npy')
    
    preds = loaded_model.predict(data)
    predictions = [i.argmax() for i in preds]
    print(predictions)
    if(predictions==[0]):
        cancer='Benign'
    elif(predictions==[1]):
        cancer='InSitu'
    elif(predictions==[2]):
        cancer='Invasive'
    else:
        cancer='Normal'
        
    
    return cancer

In [4]:
image_path=r'images/'
run_predictions(image_path)

Found 1 images belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
[0]


'Benign'